In [22]:
import re
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.chains import LLMChain
from langchain.schema import Document
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
import os

from dotenv import load_dotenv
load_dotenv()

# Initialize Google Generative AI
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

# Define the get_diabetes_level function
def get_diabetes_level(a1c):
    if a1c < 5.1:
        return "No Diabetes"
    elif 5.1 <= a1c < 7:
        return "Pre-Diabetes"
    elif a1c >= 7:
        return "Diabetes"
    else:
        return "Invalid A1C value"

# Initialize the ChatGoogleGenerativeAI model
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

# Load the PDF and extract text
pdf_path = "Basics of Diabetes Management.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pages = pdf_loader.load_and_split()

# Create context from the first few pages of the PDF
context = "\n".join(page.page_content for page in pages[:30])

# Define the prompt template
prompt_template = """
You are a diabetes instructor. Answer the question as precisely as possible using the provided context. If the answer is
not contained in the context, say "answer not available in context".

Context: \n{context}\n
Question: \n{question}\n
Answer:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Manually set up the QA chain using LLMChain
qa_chain = LLMChain(
    llm=model,
    prompt=prompt
)

# Function to chat with PDF
def chat_with_pdf(question, context):
    if "A1C" in question:
        try:
            # Extract the A1C value from the question by splitting and iterating over words
            words = question.split()
            for word in words:
                if word.replace('.', '').isdigit():
                    a1c_value = float(word)
                    print(f"Extracted A1C value: {a1c_value}")
                    return get_diabetes_level(a1c_value)
            return "A1C value not found in the question. Please provide a numerical value."
        except ValueError:
            return "Invalid input for A1C value. Please provide a numerical value."
    elif "do i have diabetes" in question:
        # Extract the A1C value from the question
        a1c_value_match = re.search(r"(\d+(\.\d+)?)", question)
        if a1c_value_match:
            a1c_value = float(a1c_value_match.group(1))
            diabetes_level = get_diabetes_level(a1c_value)
            if diabetes_level == "Diabetes" or diabetes_level == "Pre-Diabetes":
                return "Yes, you have diabetes or pre-diabetes."
            else:
                return "No, you don't have diabetes."
        else:
            return "A1C value not found in the question. Please provide a numerical value."
    else:
        # Create Document object for the context
        input_document = Document(page_content=context)
        response = qa_chain(
            {"context": context, "question": question}
        )
        return response['text']

# Example questions
questions = [
    "What is insulin therapy?",
    "Give dietary recommendations?",
    "my A1C is 8 , do i have diabetes?",
    "my A1C is 5 do i have diabetes?",
]

# Chat with PDF
for question in questions:
    answer = chat_with_pdf(question, context)
    print(f"Q: {question}\nA: {answer}\n")


Q: What is insulin therapy?
A: Insulin therapy is essential for individuals with Type 1 diabetes and sometimes necessary for Type 2 diabetes. Insulin must be injected or used with an insulin pump.

Q: Give dietary recommendations?
A: Dietary recommendations include carbohydrate management, meal planning, and avoiding sugary foods.

Extracted A1C value: 8.0
Q: my A1C is 8 , do i have diabetes?
A: Diabetes

Extracted A1C value: 5.0
Q: my A1C is 5 do i have diabetes?
A: No Diabetes



In [ ]:
#below code with user input chatbot 

In [26]:
import re
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.chains import LLMChain
from langchain.schema import Document
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
import os

from dotenv import load_dotenv
load_dotenv()

# Initialize Google Generative AI
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

# Define the get_diabetes_level function
def get_diabetes_level(a1c):
    if a1c < 5.1:
        return "No Diabetes"
    elif 5.1 <= a1c < 7:
        return "Pre-Diabetes"
    elif a1c >= 7:
        return "Diabetes"
    else:
        return "Invalid A1C value"

# Initialize the ChatGoogleGenerativeAI model
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

# Load the PDF and extract text
pdf_path = "Basics of Diabetes Management.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pages = pdf_loader.load_and_split()

# Create context from the first few pages of the PDF
context = "\n".join(page.page_content for page in pages[:30])

# Define the prompt template
prompt_template = """
You are a diabetes instructor. Answer the question as precisely as possible using the provided context. If the answer is
not contained in the context, say "answer not available in context".

Context: \n{context}\n
Question: \n{question}\n
Answer:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Manually set up the QA chain using LLMChain
qa_chain = LLMChain(
    llm=model,
    prompt=prompt
)

# Function to chat with PDF
def chat_with_pdf(question, context):
    if "A1C" in question:
        try:
            # Extract the A1C value from the question by splitting and iterating over words
            words = question.split()
            for word in words:
                if word.replace('.', '').isdigit():
                    a1c_value = float(word)
                    print(f"Extracted A1C value: {a1c_value}")
                    return get_diabetes_level(a1c_value)
            return "A1C value not found in the question. Please provide a numerical value."
        except ValueError:
            return "Invalid input for A1C value. Please provide a numerical value."
    elif "do i have diabetes" in question:
        # Extract the A1C value from the question
        a1c_value_match = re.search(r"(\d+(\.\d+)?)", question)
        if a1c_value_match:
            a1c_value = float(a1c_value_match.group(1))
            diabetes_level = get_diabetes_level(a1c_value)
            if diabetes_level == "Diabetes" or diabetes_level == "Pre-Diabetes":
                return "Yes, you have diabetes or pre-diabetes."
            else:
                return "No, you don't have diabetes."
        else:
            return "A1C value not found in the question. Please provide a numerical value."
    else:
        # Create Document object for the context
        input_document = Document(page_content=context)
        response = qa_chain(
            {"context": context, "question": question}
        )
        return response['text']

# Continuous interaction loop
while True:
    # Get user input
    user_question = input("Ask a question (type 'exit' to quit): ")

    # Check if user wants to exit
    if user_question.lower() == 'exit':
        print("Exiting...")
        break

    # Get answer using LLM
    answer = chat_with_pdf(user_question, context)
    print("Q:", user_question)
    print("A:", answer)


Q: what is insulin
A: Insulin is a hormone that helps the body use glucose for energy. It is essential for individuals with Type 1 diabetes and sometimes necessary for Type 2 diabetes.
Q: what is diabetes
A: Diabetes is a chronic condition characterized by high levels of glucose in the blood.
Exiting...
